<a href="https://colab.research.google.com/github/mrarringtonfh/FH_NewUserScripts/blob/main/FHStudentCleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive
This step connects to your Google drive - you must have access to the shared Technology drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Cleanup
The script below creates a CSV suitable for importing into AD using the ADManagerPlus applicaiton.

1. Go to Husky Hub --> Core --> Reporting --> Manage Lists
2. Choose "Manage basic and advanced lists"
3. Edit "Incoming Students" list
4. In the Filters tab, change the "on or after date" filter as needed.
5. Click "Preview"
6. Click "Export" and save the file as "NewStudents_FH.csv" in the folder ColabData in the shared Technology drive folder. Replace the existing file.






In [ ]:
import csv
import random
import string
import datetime

def get_random_password():
  # Generates a random password
  words = [line.strip() for line in open("/content/drive/Shareddrives/Technology/ColabData/membean6.txt", "r")]
  word1 = random.choice(words).capitalize()
  word2 = random.choice(words).capitalize()[0:2]
  number = random.randint(100, 999)
  special = random.choice(string.punctuation)
  return f"{word1}{special}{number}"

def process_student_data(student):
  # Processes student data and returns a dictionary
  last_name = student["LastName"].replace(" ", "")
  short_last_name = last_name if len(last_name) < 5 else last_name[:5]

  campus = "EastCampusStudents" if int(student["GradYear"]) > 2030 else "WestCampusStudents"

  building = "LSStudents" if int(student["GradYear"]) > 2030 else (
      "MSStudents" if int(student["GradYear"]) > 2028 else "USStudents"
  )

  password = get_random_password()

  return {
      "givenName": student["PreferredName"],
      "initials": student["MiddleName"][0] if student["MiddleName"] else "",
      "sn": student["LastName"],
      "password": password,
      "description": f"Class of {student['GradYear']}",
      "department": f"Class of {student['GradYear']}",
      "OUName": f"OU={student['GradYear']},OU=Students,OU=FHS-Users,DC=FLINTHILL,DC=ORG",
      "userAccountControl": "512",
      "memberOf": f'"CN=Student,OU=Groups,OU=FHS-Users,DC=FLINTHILL,DC=ORG;CN={building},OU=Groups,OU=FHS-Users,DC=FLINTHILL,DC=ORG;CN={campus},OU=Groups,OU=FHS-Users,DC=FLINTHILL,DC=ORG"',
      "gappsOrgUnitPath": f"/Students/{student['GradYear']}",
      "gappsGroupEmail": f"{student['GradYear']}@flinthill.org",
      "templateName": "FHSStudentUsers",
      "userid" : student["UserID"],
      "username": student['FirstName'][0].lower() + last_name.lower(),
      "email": f"{student['FirstName'][0].lower()}{last_name.lower()}@flinthill.org",
      "ipphone": student['FirstName'][0].lower() + short_last_name.lower(),
  }

# Read student data from CSV
with open("/content/drive/Shareddrives/Technology/ColabData/NewStudents_FH.csv") as csvfile:
  reader = csv.DictReader(csvfile)
  student_data = [process_student_data(student) for student in reader]

# Get today's date in YYYYMMDD format
today = datetime.date.today().strftime("%Y%m%d")

# Construct the output filename with date prefix
output_filename = f"FHStudentsReadyForImport_{today}.csv"

with open("/content/drive/Shareddrives/Technology/User Accounts/2024-2025/" + output_filename, "w", newline="") as csvfile:
    fieldnames = student_data[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(student_data)

print(f"Student data processed and written to {output_filename}")

Student data processed and written to FHStudentsReadyForImport_20241120.csv


# Add to new students file
Writes all students who have enrolled in the 2024-2025 school year to a csv file called NewStudents

In [1]:
import pandas as pd
import os

def filter_and_write(input_file, output_file, filter_condition):
    """
    Filters data from an input CSV file based on a given condition and writes the filtered data to a new output CSV file.

    Args:
        input_file: Path to the input CSV file.
        output_file: Path to the output CSV file.
        filter_condition: A boolean expression to filter the data.
    """

    try:
        # Read the original CSV, including the header
        df = pd.read_csv(input_file)

        # Ensure the 'Student ID' column is treated as a string directly on the DataFrame
        df['Student ID'] = df['Student ID'].astype(int)

        # Filter the data
        filtered_df = df[filter_condition]

        # Write the filtered data to the output CSV, including the header
        filtered_df.to_csv(output_file, index=False)

    except FileNotFoundError:
        print("Input or output file not found.")
    except PermissionError:
        print("Permission denied to access files.")

# Example usage:
input_file = "/content/drive/Shareddrives/Technology/User Accounts/Student Account Information/StudentMain/AllStudents.csv"
output_csv = "/content/drive/Shareddrives/Technology/User Accounts/Student Account Information/StudentMain/NewStudents.csv"

filter_and_write(input_file, output_csv, df['School Year Enrolled'] == '2024-2025')

NameError: name 'df' is not defined

# Encrypt

In [28]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789()$&@?!"
cryptabet = "JL$3frKkh1j2VvHBP&CEs7@tz!l9SdXFYayMcTbogx0wI(DpneG6mO5NZQ)Wqi?R8U4Au"

def scramble(msg):
  """Encrypts a message using a character mapping."""
  mapping = str.maketrans(alphabet, cryptabet)
  return msg.translate(mapping)

def unscramble(code):
  """Decrypts a message using a character mapping."""
  mapping = str.maketrans(cryptabet, alphabet)
  return code.translate(mapping)

message = "Diablo%809"
encrypted_message = scramble(message)
print(encrypted_message)  # Output: TBPPR, ZLFRR!

decrypted_message = unscramble(encrypted_message)
print(decrypted_message)  # Output: Hello, world!

3yl9Tg%qmi
Diablo%809


In [31]:
def read_cipher_from_file(filename):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            alphabet = lines[0].strip().split(': ')[1]
            cryptabet = lines[1].strip().split(': ')[1]
            return alphabet, cryptabet
    except (FileNotFoundError, IndexError) as e:
        print(f"Error reading cipher file: {e}")
        return None, None

def scramble(msg, alphabet, cryptabet):
    mapping = str.maketrans(alphabet, cryptabet)
    return msg.translate(mapping)

def unscramble(code, alphabet, cryptabet):
    mapping = str.maketrans(cryptabet, alphabet)
    return code.translate(mapping)

# Assuming the cipher file is in the same directory as your script
filename = "/content/drive/Shareddrives/Technology/User Accounts/Student Account Information/StudentMain/cipher.txt"
alphabet, cryptabet = read_cipher_from_file(filename)

message = "Diablo%809"
encrypted_message = scramble(message, alphabet, cryptabet)
print(encrypted_message)

decrypted_message = unscramble(encrypted_message, alphabet, cryptabet)
print(decrypted_message)

3yl9Tg%qmi
Diablo%809


In [23]:
import pandas as pd

def read_cipher_from_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        alphabet = lines[0].strip().split(': ')[1]
        cryptabet = lines[1].strip().split(': ')[1]
        return alphabet, cryptabet

def scramble(msg, alphabet, cryptabet):
    mapping = str.maketrans(alphabet, cryptabet)
    return msg.translate(mapping)

def unscramble(code, alphabet, cryptabet):
    mapping = str.maketrans(cryptabet, alphabet)
    return code.translate(mapping)

# Read the cipher from the file
filename = "/content/drive/Shareddrives/Technology/User Accounts/Student Account Information/StudentMain/cipher.txt"
alphabet, cryptabet = read_cipher_from_file(filename)

# Read the CSV file
df = pd.read_csv("/content/drive/Shareddrives/Technology/User Accounts/Student Account Information/StudentMain/AllStudents.csv")

# Apply the encryption/decryption function to a specific column
df['encrypted_column'] = df['Student Pwd'].apply(lambda x: scramble(x, alphabet, cryptabet))

# Write the new CSV file
df.to_csv("/content/drive/Shareddrives/Technology/User Accounts/Student Account Information/StudentMain/encrypted_output.csv", index=False)